In [1]:
!pwd

/home/aman/dl


In [2]:
import json
import numpy as np
squad_data = None
with open("../squad/dataset/train-v1.1.json", "r") as f:
    squad_data = json.loads(f.read())

In [3]:
import pprint

In [4]:
data = squad_data["data"]
ignore_parts = ["#", ",", ".",  "(", ")", "'", "\""]
from pytorch_pretrained_bert import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
all_datapoints = []
for doc in data:
    for paragraph in doc["paragraphs"]:
        text = paragraph["context"]
        tokens = text.split()
        token_len = len(tokens)
        tokens = " ".join(tokens)
        b_tokens = tokenizer.tokenize(tokens)
        tokenized = " ".join(b_tokens)
        for qas in paragraph["qas"]:
            question = tokenizer.tokenize(qas["question"])
            #print(question)
            for answer in qas["answers"]:
                try:
                    t_answer = tokenizer.tokenize(answer["text"])
                    answer_start = tokenized.index(" ".join(t_answer))
                    _t = tokenized[:answer_start].split()
                    d_t = tokenized.split()
                except:
                    pprint.pprint(qas, indent=4)
                if (len(d_t) <= 512):
                    add = True
                    try:
                        _start = len(_t)
                        _end = _start + len(t_answer)
                        #print(b_tokens[_start:_end])
                        data_tokens = ["[CLS]"]
                        data_tokens.extend(question)
                        data_tokens.append("[SEP]")
                        data_tokens.extend(b_tokens)
                        data_tokens.append("[SEP]")
                        _start = _start + len(question) + 2 
                        _end = _end + len(question) + 2 
                        #print(data_tokens, _start, _end, "\n", data_tokens[_start:_end])
                        while (len(data_tokens) < 512):
                            data_tokens.append("[PAD]")
                        sentence_types = np.ones((512,), dtype=np.int8)
                        sentence_types[:len(question) + 2] = 0    
                        data_tokens = data_tokens[:512]
                        dp = {
                            "doc":b_tokens,
                            #"doc_tokens":doc_token_ids,
                            "question":question,
                            #"question_tokens":question_token_ids,
                            "answer_start":_start,
                            "answer_end":_end,
                            "data":{
                                "tokens":tokenizer.convert_tokens_to_ids(data_tokens),
                                "segments":sentence_types
                            }
                        }
                    except:
                        add = False
                    if (add):
                        all_datapoints.append(dp)

{   'answers': [{'answer_start': 176, 'text': 'rld around it'}],
    'id': '57309ede396df9190009621b',
    'question': 'What makes up the sum of relations to an entity?'}
{   'answers': [{'answer_start': 58, 'text': 'kno'}],
    'id': '56df865956340a1900b29ceb',
    'question': 'Would you consider aesthetic elements alone in architectural '
                'lighting design?'}
{   'answers': [{'answer_start': 156, 'text': 'ano'}],
    'id': '56df95d44a1a83140091eb81',
    'question': 'Would a lower GAI mean higher apparent saturation or '
                'vividness of object colors?'}
{   'answers': [{'answer_start': 223, 'text': '30s'}],
    'id': '56df736f5ca0a614008f9a91',
    'question': 'How many pubs existed on Union Street in the 1930s?'}
{   'answers': [{'answer_start': 29, 'text': '250 million'}],
    'id': '56e065a0231d4119001ac08d',
    'question': "How much money was spent on construction of the island's "
                'airport?'}
{   'answers': [{'answer_start': 202, 'te

In [5]:
import pickle
pickle.dump(all_datapoints, open("./squad_processed_1.1.pickle", "wb"))